In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import torch
import cv2 
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import torchvision
import wandb
from time import time

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

from Trainer import train_model
from Dataloader import JetcarDataset, SimulatorDataset 
from Models import NetworkNvidia, ResNet18, ResNet34, AlexNet, VGG16

In [4]:
transforms = torch.nn.Sequential(
    T.RandomResizedCrop(size=(224, 224),scale=(0.9,1)),
    T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 4)),
    T.ColorJitter(brightness=.3, hue=.3, saturation=.3),
    #T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
) 

In [5]:
realData = JetcarDataset("data_mix/data.csv", "data_mix/img/", transform=transforms)

In [ ]:
sweep_config = {
    'method': 'random', 
    'metrics':{'name':'loss','goal':'minimize'}
    }

parameters_dict = {
    'epochs':{'value':100},
    'optimizer': {'values': ['sgd','adam']}, 
    'scheduler': {'values': [True, False]},
    'batch_size':{'values':[8, 16, 32, 64]},
    'learning_rate': {'values': [0.1, 0.05, 0.001, 0.0005]}, # 4 combinations
    'model':{'values': ['Nvidia', 'ResNet18', 'ResNet34', 'AlexNet', 'Vgg16']},
    'loss':{'values': ['MSE', 'L1']}
    }

sweep_config['parameters'] = parameters_dict

print(sweep_config)

In [1]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(), lr=learning_rate)
    elif optimizer == "adamax":
        optimizer = optim.Adamax(network.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    elif optimizer == "rmsprop":
        optimizer = optim.RMSprop(network.parameters(), lr=learning_rate, alpha=0.99, eps=1e-08, weight_decay=0,
                                  momentum=0, centered=False)
    return optimizer

In [2]:
# choose network
def build_network(model):
    if model == 'Nvidia':
        network = NetworkNvidia()
    if model == 'ResNet18':
        network = ResNet18()
    if model == 'ResNet34':
        network = ResNet34() 
    if model == 'AlexNet':
        network = AlexNet()
    if model == 'Vgg16':
        network = VGG16()
    return network
    

In [ ]:
def build_loss(loss):
    if loss=='MSE':
        criterion = nn.MSELoss()
    if loss=='L1':
        criterion = nn.L1Loss()
    return criterion

In [ ]:
def sweep_train(n_epochs, model, train_loader, criterion, optimizer, device = 'cpu', scheduler=False):
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = n_epochs)
    #wandb.watch(model, loss_fn, log='all', log_freq=1000, log_graph=True)
    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], n_epochs, device))

    model = model.to(device)
    total_params = sum(p.numel() for p in model.parameters())
    wandb.log({'total_params':total_params})
    
    # Training Loop
    start_time = time()
    for epoch in range(1, n_epochs + 1):
        epoch_start_time = time()
        train_loss = 0
        total_train = 0
        # train model
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            total_train += target.shape[0]

        if scheduler != False:
            # update scheduler
            scheduler.step()

        # compute average loss
        train_loss /= total_train
        epoch_end_time = time()
        
        print('Epoch: {}/{} \tTrain Loss: {:.6f} \tEpoch Time:{:.6f}' .format(epoch, n_epochs, train_loss, epoch_end_time-epoch_start_time))
        wandb.log({'epoch':epoch, 'train_loss':train_loss, 'time_per_epoch':epoch_end_time-epoch_start_time})  
        
    end_time = time()
    wandb.log({'total_time':end_time-start_time})
    

In [ ]:
# wandb.init(project="self-driving-car-model", entity="sudharsanananth")

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="self-driving-car-model", entity="sudharsanananth")

In [ ]:
def sweep(config=None):
    
    # Initialize a new wandb run
    with wandb.init(config=config):
        # this config will be set by sweep controller, randomly assigned each time
        config = wandb.config
        
        trainloader = DataLoader(realData, batch_size=config.batch_size, shuffle=True, num_workers=8)
        network = build_network(config.model)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        loss = build_loss(config.loss)
        sweep_train(config.epochs, network, trainloader, loss, optimizer, "cuda", config.scheduler)

In [ ]:
wandb.agent(sweep_id, sweep, count=20)